In [1]:
import os

In [2]:
path = "../input/kla1task2/output"

In [3]:
labels =os.listdir(path)

In [4]:
d = {}
for x in labels :
    d[x] = len(os.listdir(path+'/'+ x))

In [5]:
len(d)

In [6]:
d

In [7]:
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import matplotlib.pyplot as plt


In [8]:

filenames = os.listdir("../input/kla1task2/output")


In [9]:
files = []
categories = []
for filename in filenames:
    
    file  = "../input/kla1task2/output/" + filename
    
    for x in os.listdir(file):
        p = file +"//"+ x
        files.append(p)
        categories.append(filename)
#         print(filename,p)
    
df = pd.DataFrame({
    'filename': files,
    'category': categories
})

In [10]:
df.head()

In [11]:
FAST_RUN = False
IMAGE_WIDTH=256
IMAGE_HEIGHT=256

In [12]:
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=1

In [13]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate,MaxPooling2D,Dropout,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, Callback



In [22]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import layers

In [15]:
def identity_block(input_tensor, kernel_size, filters):
    
    filters1, filters2, filters3 = filters
    
    

    x = Conv2D(filters1,1)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, 1)(x)
    x = BatchNormalization()(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

In [23]:
def conv_block(input_tensor, kernel_size, filters, strides=(2, 2)):
    
    
    filters1, filters2, filters3 = filters

    x = Conv2D(filters1, 1, strides=strides)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3,1)(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(filters3,  1, strides=strides)(input_tensor)
    shortcut = BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [24]:
inputs = Input(shape=(224,224,1))

x = ZeroPadding2D((3, 3))(inputs)
x = Conv2D(64, (7, 7), strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, 3, [64, 64, 256],  strides=(1, 1))
x = identity_block(x, 3, [64, 64, 256] )
x = identity_block(x, 3, [64, 64, 256])

x = conv_block(x, 3, [128, 128, 512])
x = identity_block(x, 3, [128, 128, 512])
x = identity_block(x, 3, [128, 128, 512])
x = identity_block(x, 3, [128, 128, 512])

x = conv_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])
x = identity_block(x, 3, [256, 256, 1024])

x = conv_block(x, 3, [512, 512, 2048])
x = identity_block(x, 3, [512, 512, 2048])
x = identity_block(x, 3, [512, 512, 2048])

x = AveragePooling2D((7, 7))(x)

   
x = Flatten()(x)
x = Dense(28, activation='softmax')(x)

resmodel = Model(inputs, x)
resmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
resmodel.summary()

In [26]:
from sklearn.model_selection import train_test_split
df = df.sample(frac = 1)

plt.figure(figsize = (20,20))
df['category'].value_counts().plot.bar()

In [27]:
total_train = df.shape[0]
total_validate = df.shape[0]
batch_size = 64

In [28]:
t_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

t_generator = t_datagen.flow_from_dataframe(
    df, 
    color_mode="grayscale",
    x_col='filename',
    y_col='category',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=batch_size
)


In [29]:
t_generator.class_indices

In [30]:
len(d)

In [31]:
d = t_generator.class_indices


In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
red = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

In [ ]:
epochs=100
modelhistory = resmodel.fit(
    t_generator, 
    epochs=epochs,
    
    steps_per_epoch=len(df)//batch_size,callbacks=[red, checkpoint])
    


In [ ]:
resmodel.save_weights("task2(w).h5")
resmodel.save("task2(m).h5")